# Hands-on: Setting Closed Book Baseline

## Installation

In [2]:
!pip install datasets evaluate transformers accelerate bitsandbytes sentencepiece

## Imports

In [3]:
import random, torch, evaluate, json
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from huggingface_hub import login
login(token='')

## Load Data

In [ ]:
ds = load_dataset("hotpot_qa", "distractor", split="train[:200]")
questions = ds["question"][:25]
gold_answers = ds["answer"][:25]

In [ ]:
ds.keys()

In [ ]:
questions[0]

In [ ]:
gold_answers[0]

## Load Model

In [ ]:
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model      = AutoModelForCausalLM.from_pretrained(
                 model_name, device_map="auto", torch_dtype=torch.float16)

generator = pipeline("text-generation", model=model, tokenizer=tokenizer,
                     temperature=0.1,
                     max_new_tokens=128)

## Closed-book LLM Prediction

In [ ]:
predictions = []

for q in questions:
    prompt = ( "You are an expert question-answering system.\n"
               f"Question: {q}\n"
               "Answer briefly:\n" )
    ans = generator(prompt)[0]["generated_text"].split("Answer briefly:\n")[-1]
    print(f"{q} -> {ans}")
    predictions.append(ans.strip())


In [ ]:
squad = evaluate.load("squad")
results = squad.compute(predictions=predictions, references=gold_answers)
print(json.dumps(results, indent=2))